In [1]:
import pandas as pd
import numpy as np
import os,sys
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/25_KingsOfML')
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# df_train=pd.read_csv('01.RawData/train.csv')
# df_test=pd.read_csv('01.RawData/test_BDIfz5B.csv')
# df_cmpgn=pd.read_csv('01.RawData/campaign_data.csv')
# df_sample=pd.read_csv('01.RawData/sample_submission_4fcZwvQ.csv')
# df_train['send_date']=pd.to_datetime(df_train['send_date'],format='%d-%m-%Y %H:%M')
# df_test['send_date']=pd.to_datetime(df_test['send_date'],format='%d-%m-%Y %H:%M')
# df_train.sort_values(by=['user_id','send_date'],inplace=True)
# df_test.sort_values(by=['user_id','send_date'],inplace=True)
# store=pd.HDFStore('01.RawData/DataStore.h5')
# store['train']=df_train
# store['test']=df_test
# store['cmpgn']=df_cmpgn
# store['sample']=df_sample
# store.close()

df_train=pd.read_hdf('01.RawData/DataStore.h5', 'train')
df_test=pd.read_hdf('01.RawData/DataStore.h5', 'test')
df_cmpgn=pd.read_hdf('01.RawData/DataStore.h5', 'cmpgn')
df_sample=pd.read_hdf('01.RawData/DataStore.h5', 'sample')

In [15]:
comm_type=df_cmpgn.set_index('campaign_id')
comm_type['communication_type']=['communication_type'].astype('category').cat.codes

df_train['month']=df_train.send_date.dt.month
df_train['rcv']=1
df_train['communication_type']=df_train['campaign_id'].map(comm_type)
signals_1=df_train.groupby(['user_id','month'])[['rcv','is_click','is_open']].sum().add_prefix('sum_')
signals_1=signals_1.stack().unstack(1).fillna(0).cumsum(axis=1).stack().unstack(1).reset_index()
signals_1['month']=signals_1['month']+1
signals_2=df_train.groupby(['user_id','communication_type','month'])[['rcv','is_click','is_open']].sum().add_prefix('sum_')
signals_2=signals_2.stack().unstack(2).fillna(0).cumsum(axis=1).stack().unstack([2,1]).fillna(0)
signals_2.columns=signals_2.columns.get_level_values(0)+'_'+signals_2.columns.get_level_values(1).astype(np.str)
signals_2=signals_2.reset_index()
signals_2['month']=signals_2['month']+1
signals_1=pd.merge(signals_1,signals_2,on=['user_id','month'])
signals_2=df_train.groupby(['user_id','communication_type','month'])[['rcv','is_click','is_open']].sum().add_prefix('sum_ct_')
signals_2=signals_2.stack().unstack(2).fillna(0).cumsum(axis=1).stack().unstack(2).reset_index()
signals_2['month']=signals_2['month']+1

In [90]:
df_train['source']='train'
df_train['month']=df_train['send_date'].dt.month
df_test['source']='test'
df_test['month']=13
master=pd.concat([df_train,df_test])
master=pd.merge(master,signals_1,on=['user_id','month'],how='left').fillna(0)
master=pd.merge(master,signals_2,on=['user_id','communication_type','month'],how='left').fillna(0)
master['campaign_rank']=master.groupby('user_id')['campaign_id'].rank()
master=pd.concat([master,pd.get_dummies(master['communication_type'].astype(np.int64),prefix='com')],axis=1)
master.set_index(['user_id','campaign_id','id'],inplace=True)

sig_sets=[x for x in master.columns if 'sum_rcv' in x]
for sig in sig_sets:
    master[sig.replace('rcv','open_rate')]=1.0*master[sig.replace('rcv','is_open')]/(master[sig.replace('rcv','rcv')]+0.0001)
    master[sig.replace('rcv','click_rate')]=1.0*master[sig.replace('rcv','is_click')]/(master[sig.replace('rcv','rcv')]+0.0001)
    master[sig.replace('rcv','click_per_open')]=1.0*master[sig.replace('rcv','is_click')]/(master[sig.replace('rcv','is_open')]+0.0001)

In [97]:
def get_train_test_campaigns(valid=True):
    train_months=[8,9] if valid==True else [8,9,10,11,12]
    valid_months=[10,11,12] if valid==True else [13]
    train_campaigns=master[master.month.isin(train_months)].index.get_level_values(1).drop_duplicates()
    valid_campaigns=master[master.month.isin(valid_months)].index.get_level_values(1).drop_duplicates()
    return train_campaigns.tolist(),valid_campaigns.tolist()
  
train_campaigns,valid_campaigns=get_train_test_campaigns(valid=True)
master=master[master.index.get_level_values(1).isin(train_campaigns+valid_campaigns)]
master['ds']=np.where(master.index.get_level_values(1).isin(train_campaigns),'1.Train','2.Valid')
master.sort_values(by='ds',inplace=True)

In [62]:
#Get campaign level signals
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(binary=True)
train_campaigns=train_X.reset_index()['campaign_id'].drop_duplicates()
valid_campaigns=valid_X.reset_index()['campaign_id'].drop_duplicates()
vocab_train=cv.fit(df_cmpgn[df_cmpgn['campaign_id'].isin(train_campaigns)]['subject']).vocabulary_.keys()
vocab_valid=cv.fit(df_cmpgn[df_cmpgn['campaign_id'].isin(valid_campaigns)]['subject']).vocabulary_.keys()
vocab_subject=list(set(vocab_train).intersection(set(vocab_valid)))
vocab_train=cv.fit(df_cmpgn[df_cmpgn['campaign_id'].isin(train_campaigns)]['email_body']).vocabulary_.keys()
vocab_valid=cv.fit(df_cmpgn[df_cmpgn['campaign_id'].isin(valid_campaigns)]['email_body']).vocabulary_.keys()
vocab_email=list(set(vocab_train).intersection(set(vocab_valid)))



In [111]:
import re,string

def get_free_text_vars(df,var,label):
    df['sig_ft_0_%s'%label]=df[var].astype(np.str).str.len()                                                             # Num characters
    df['sig_ft_1_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[%s]'%string.printable,'',x)).str.len()         # Num non printable chars
    df['sig_ft_2_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^#$%&*:;<=>?@\^_`|~]','',x)).str.len()         # Num Special Characters
    df['sig_ft_3_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^0-9]','',x)).str.len()                        # Num numerals
    df['sig_ft_4_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^a-z]','',x)).str.len()                        # Num lower case
    df['sig_ft_5_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^A-Z]','',x)).str.len()                        # Num upper case
    df['sig_ft_6_%s'%label]=df[var].astype(np.str).map(lambda x: re.sub('[^ ]','',x)).str.len()                          # Num spaces
    return df

cmpgn_sigs=df_cmpgn.set_index(['campaign_id'])
cmpgn_sigs=get_free_text_vars(cmpgn_sigs,'subject','sub')
cmpgn_sigs=get_free_text_vars(cmpgn_sigs,'email_body','email')
cv=CountVectorizer(binary=True,vocabulary=vocab_subject)
bow_1=cv.fit_transform(cmpgn_sigs['subject']).todense()
bow_1=pd.DataFrame(bow_1,columns=['sig_sub_%s'%x for x in vocab_subject],index=cmpgn_sigs.index.tolist())
cv=CountVectorizer(binary=True,vocabulary=vocab_email)
bow_2=cv.fit_transform(cmpgn_sigs['email_body']).todense()
bow_2=pd.DataFrame(bow_2,columns=['sig_email_%s'%x for x in vocab_email],index=cmpgn_sigs.index.tolist())
cmpgn_sigs=pd.concat([cmpgn_sigs,pd.get_dummies(cmpgn_sigs['communication_type'],prefix='sig_com'),bow_1,bow_2],axis=1)
cmpgn_sigs.rename(columns={'total_links':'sig_links1','no_of_internal_links':'sig_links2',
        'no_of_images':'sig_images','no_of_sections':'sig_sections'},inplace=True)
cmpgn_sigs=cmpgn_sigs[[x for x in cmpgn_sigs.columns if x[:3]=='sig']].to_sparse()
cmpgn_sigs=pd.merge(master.index.get_level_values(1).to_series().to_frame(),cmpgn_sigs,left_index=True,right_index=True)
cmpgn_sigs=cmpgn_sigs[[x for x in cmpgn_sigs.columns if x[:3]=='sig']].to_sparse()

In [127]:
from scipy.sparse import coo_matrix,csr_matrix,hstack
signals=[x for x in master.columns if x[:3] in ['sum','com'] or x in ['campaign_rank']]
X_user_sigs=csr_matrix(master[signals].values)
X_cmpgn_sigs=csr_matrix(cmpgn_sigs.values)
X_user_dummies=master.index.get_level_values(0).to_series().astype('category').cat.codes
X_user_dummies=coo_matrix((np.ones((X_user_dummies.shape)),(np.arange(len(X_user_dummies)),X_user_dummies)))
X=csr_matrix(hstack((X_user_sigs,X_cmpgn_sigs,X_user_dummies)))
Y=master['is_click']

train_X=X[:counts]
valid_X=X[counts:]
train_Y=Y[:counts]
valid_Y=Y[counts:]

In [145]:
print [x.shape for x in [train_X,valid_X,train_Y,valid_Y]]

[(407164, 167572), (356582, 167572), (407164L,), (356582L,)]


In [147]:
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

lgb_train = lgb.Dataset(train_X, train_Y)
lgb_test = lgb.Dataset(valid_X, valid_Y)
watchlist = [lgb_train,lgb_test]

for param in [3,5,10]:
    params = {
        'learning_rate': 0.04,
        'max_depth':param,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 2**5-1,
        'verbose': -1,
        'data_random_seed': 1,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'feature_fraction': 0.6,
        'min_data_in_leaf': 250,
    }
    
    gbm = lgb.train(params,lgb_train,num_boost_round=50,valid_sets=watchlist,verbose_eval=10)
    print (param,roc_auc_score(train_Y, gbm.predict(train_X)))
    print (param,roc_auc_score(valid_Y, gbm.predict(valid_X)))

[10]	training's auc: 0.695945	valid_1's auc: 0.655417
[20]	training's auc: 0.697329	valid_1's auc: 0.660727
[30]	training's auc: 0.698973	valid_1's auc: 0.657074
[40]	training's auc: 0.699579	valid_1's auc: 0.656064
[50]	training's auc: 0.699691	valid_1's auc: 0.656473
(3, 0.69969063178406554)
(3, 0.65647291715009859)
[10]	training's auc: 0.714288	valid_1's auc: 0.6565
[20]	training's auc: 0.717214	valid_1's auc: 0.655714
[30]	training's auc: 0.71471	valid_1's auc: 0.654337
[40]	training's auc: 0.715157	valid_1's auc: 0.654623
[50]	training's auc: 0.715641	valid_1's auc: 0.656455
(5, 0.71564061999816031)
(5, 0.65645492896236124)
[10]	training's auc: 0.719763	valid_1's auc: 0.654531
[20]	training's auc: 0.720542	valid_1's auc: 0.646745
[30]	training's auc: 0.720658	valid_1's auc: 0.646161
[40]	training's auc: 0.720949	valid_1's auc: 0.647588
[50]	training's auc: 0.722045	valid_1's auc: 0.64803
(10, 0.72204541419118551)
(10, 0.64802959300334162)


In [44]:
valid_Y['pred']=gbm.predict(valid_X)


In [45]:
valid_Y.groupby(valid_X.index.get_level_values(1)).mean()

,is_click,is_open,pred
campaign_id,,,
48,0.012166,0.051306,0.487938
50,0.014455,0.049232,0.487959
51,0.043792,0.314529,0.488435
52,0.014192,0.048588,0.487940
53,0.010090,0.036544,0.487961
54,0.010251,0.201196,0.487946


In [151]:
# %run 00.Scripts/02.LGBMBase.py

[5]	training's auc: 0.670657
[10]	training's auc: 0.673225
[15]	training's auc: 0.673811
[20]	training's auc: 0.674385
[25]	training's auc: 0.67453
[5]	training's auc: 0.740107
[10]	training's auc: 0.744053
[15]	training's auc: 0.745894
[20]	training's auc: 0.745019
[25]	training's auc: 0.750529


In [47]:
pd.Series(gbm.feature_importance(),index=train_X.columns).sort_values(ascending=False)

sum_click_per_open      22
sum_rcv                 20
sum_is_click            19
sum_open_rate           19
sum_is_open             16
communication_type      14
com_2                   14
campaign_rank            6
sum_click_rate           4
com_0                    4
sum_rcv_0                3
sum_is_open_0            2
sum_open_rate_6          2
sum_ct_rcv               1
sum_click_rate_0         1
sum_is_click_3           1
sum_rcv_6                1
com_6                    1
sum_click_rate_2         0
sum_rcv_2                0
sum_is_click_4           0
sum_rcv_4                0
sum_is_open_6            0
sum_is_click_6           0
sum_is_open_2            0
sum_is_click_2           0
sum_rcv_5                0
sum_is_open_5            0
sum_is_click_5           0
sum_rcv_1                0
sum_is_open_3            0
sum_rcv_3                0
sum_is_click_0           0
sum_click_rate_4         0
sum_click_per_open_4     0
sum_open_rate_1          0
sum_is_open_4            0
s

In [146]:
?pd.get_dummies

In [137]:
print valid_X['is_click'].shape
print valid_X['is_click'].describe()

(773858L,)
count    773858.000000
mean          0.207952
std           0.058373
min           0.175426
25%           0.175426
50%           0.196661
75%           0.218063
max           0.604946
Name: is_click, dtype: float64
